In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ardl import ARDL
import statsmodels


from sklearn.metrics import mean_squared_error
from math import sqrt
import math

import pmdarima

In [2]:
# holt winters 
# single exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
# double and triple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Read Data

In [3]:
file = "../Codes/df_spot_rates.csv"
df_ecb = pd.read_csv(file)
df_ecb['TIME_PERIOD'] = pd.to_datetime(df_ecb['TIME_PERIOD'])

In [4]:
df_ecb

,TIME_PERIOD,MAT,OBS_VALUE
0,2004-09-06,3,2.034172
1,2004-09-06,4,2.063283
2,2004-09-06,5,2.092544
3,2004-09-06,6,2.121920
4,2004-09-06,7,2.151377
...,...,...,...
1783335,2023-12-29,356,2.351581
1783336,2023-12-29,357,2.350594
1783337,2023-12-29,358,2.349600
1783338,2023-12-29,359,2.348600


In [13]:
file = "../data/treasury-par-yield-curve-rates.csv"

df = pd.read_csv(file, header=0)
df['Date'] = pd.to_datetime(df['Date'])

/var/folders/gp/1_dqbwsn08sbtf7s0k5q2lsc0000gn/T/ipykernel_22357/53638615.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [14]:
df_orig = df.copy()

In [15]:
df

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2023-12-29,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,2023-12-28,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,2023-12-27,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,2023-12-26,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,2023-12-22,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8502,1990-01-08,NaN,NaN,7.79,NaN,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
8503,1990-01-05,NaN,NaN,7.79,NaN,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
8504,1990-01-04,NaN,NaN,7.84,NaN,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
8505,1990-01-03,NaN,NaN,7.89,NaN,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [16]:
df['Date']

0      2023-12-29
1      2023-12-28
2      2023-12-27
3      2023-12-26
4      2023-12-22
          ...    
8502   1990-01-08
8503   1990-01-05
8504   1990-01-04
8505   1990-01-03
8506   1990-01-02
Name: Date, Length: 8507, dtype: datetime64[ns]

In [17]:
df = df.replace("-", np.NaN)

In [27]:
cols = ['1 Mo', '3 Mo', "6 Mo", "1 Yr", "2 Yr",	"3 Yr",	"5 Yr",	"7 Yr",	"10 Yr", "20 Yr", "30 Yr"]
df = df[["Date"] + cols]

In [28]:
df = df.dropna()

In [29]:
df

,Date,1 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2023-12-29,5.60,5.40,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,2023-12-28,5.57,5.45,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,2023-12-27,5.55,5.44,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,2023-12-26,5.53,5.45,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,2023-12-22,5.54,5.44,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05
...,...,...,...,...,...,...,...,...,...,...,...,...
5603,2001-08-06,3.62,3.52,3.47,3.56,3.88,4.17,4.71,4.99,5.19,5.70,5.59
5604,2001-08-03,3.63,3.52,3.47,3.57,3.91,4.22,4.72,4.99,5.20,5.70,5.59
5605,2001-08-02,3.65,3.53,3.46,3.57,3.89,4.17,4.69,4.97,5.17,5.68,5.57
5606,2001-08-01,3.65,3.53,3.47,3.56,3.83,4.09,4.62,4.90,5.11,5.63,5.53


In [79]:
# Check if all dates are present
ls = df['Date']
for idx, ele in enumerate(ls):
    if idx == 0:
        continue

    prev_date = ls.iloc[idx - 1]

    if (ele.month - prev_date.month) != 1:
        if ele.month == 1 and prev_date.month == 12 and (ele.year - prev_date.year) == 1:
            continue
        print(f"{idx}: {ele} {prev_date}")

1: 2007-11-07 00:00:00 2007-11-06 00:00:00
2: 2007-11-08 00:00:00 2007-11-07 00:00:00
3: 2007-11-09 00:00:00 2007-11-08 00:00:00
4: 2007-11-12 00:00:00 2007-11-09 00:00:00
5: 2007-11-13 00:00:00 2007-11-12 00:00:00
6: 2007-11-14 00:00:00 2007-11-13 00:00:00
7: 2007-11-15 00:00:00 2007-11-14 00:00:00
8: 2007-11-16 00:00:00 2007-11-15 00:00:00
9: 2007-11-19 00:00:00 2007-11-16 00:00:00
10: 2007-11-20 00:00:00 2007-11-19 00:00:00
11: 2007-11-21 00:00:00 2007-11-20 00:00:00
12: 2007-11-22 00:00:00 2007-11-21 00:00:00
13: 2007-11-26 00:00:00 2007-11-22 00:00:00
14: 2007-11-27 00:00:00 2007-11-26 00:00:00
15: 2007-11-28 00:00:00 2007-11-27 00:00:00
16: 2007-11-29 00:00:00 2007-11-28 00:00:00
17: 2007-11-30 00:00:00 2007-11-29 00:00:00
19: 2007-12-04 00:00:00 2007-12-03 00:00:00
20: 2007-12-05 00:00:00 2007-12-04 00:00:00
21: 2007-12-06 00:00:00 2007-12-05 00:00:00
22: 2007-12-07 00:00:00 2007-12-06 00:00:00
23: 2007-12-10 00:00:00 2007-12-07 00:00:00
24: 2007-12-11 00:00:00 2007-12-10 00:00:

In [30]:
# Check if all dates are present
ls = df_orig.iloc[:4481]['Date']
checker = set(df['Date'])
for idx, ele in enumerate(ls):
    # date = ls.iloc[idx]
    if ele not in checker:
        print(f"{idx}: {ele} ")    

    # if idx == 0:
    #     continue

    # prev_date = ls.iloc[idx - 1]

    # if (ele.month - prev_date.month) != 1:
    #     if ele.month == 1 and prev_date.month == 12 and (ele.year - prev_date.year) == 1:
    #         continue
    #     print(f"{idx}: {ele} {prev_date}")

3307: 2010-10-11 00:00:00 
3757: 2008-12-24 00:00:00 
3761: 2008-12-18 00:00:00 
3767: 2008-12-10 00:00:00 
4479: 2006-02-08 00:00:00 
4480: 2006-02-07 00:00:00 


## Data Transformation

In [31]:
df_trans = df.copy()

In [32]:
df_trans = df_trans.rename(columns={"Date": "TIME_PERIOD"})

In [33]:
df_trans = df_trans.melt(id_vars=['TIME_PERIOD'], value_vars=['1 Mo', '3 Mo', "6 Mo", "1 Yr", "2 Yr",	"3 Yr",	"5 Yr",	"7 Yr",	"10 Yr", "20 Yr", "30 Yr"],
              var_name='MAT', value_name='OBS_VALUE'
              )

In [34]:
di = {
    '1 Mo':1, '3 Mo':3, "6 Mo":6, "1 Yr":12, "2 Yr":24,	"3 Yr":36,	"5 Yr":60,	"7 Yr":84,	"10 Yr":120, "20 Yr":240, "30 Yr":360
}

In [35]:
df_trans['MAT'] = df_trans['MAT'].replace(di)

In [36]:
df_trans = df_trans.sort_values(by=['TIME_PERIOD', "MAT"])

In [37]:
df_trans

,TIME_PERIOD,MAT,OBS_VALUE
4609,2001-07-31,1,3.67
9219,2001-07-31,3,3.54
13829,2001-07-31,6,3.47
18439,2001-07-31,12,3.53
23049,2001-07-31,24,3.79
...,...,...,...
27660,2023-12-29,60,3.84
32270,2023-12-29,84,3.88
36880,2023-12-29,120,3.88
41490,2023-12-29,240,4.20


In [38]:
# Save csv
df_trans.to_csv("../data/df_us_spot_rates.csv", index=False)

# Data Transformation